# 4. 오차역전파법(Backward Propagation) 

- 연쇄법칙에 기초한 오차역전파법을 계산그래프로 설명한 이론적인 내용은 책 Deep Learning from Scratch 5장 참조할 것
- 수치 미분은 간단하지만, 학습에 이용하기에는 계산시간이 오래 걸린다.
- 오차역전파법을 이용하면 각 매개변수에 대한 손실함수의 미분값을 훨씬 빠르게 구할 수 있다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 곱셈 노드에서의 순전파와 역전파
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

In [3]:
# 덧셈 노드에서의 순전파와 역전파
# 역전파의 입력이 그대로 출력이 된다.
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [4]:
# ReLU 노드에서의 순전파와 역전파
# 순전파에서의 입력이 0보다 작거나 같았을때, 역전파에서의 출력은 0이 된다.
class Relu:
    def __init_(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [5]:
# Sigmoid 노드에서의 순전파와 역전파
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [6]:
# Affine 계층
# 직접 손으로 풀어서 계산해서 이해해볼 것!
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응 (3차 이상 다차원텐서에 대비한 것 ex:MNIST)
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        # self.original_x_shape 앞의 "*" 는 unpacking 기능! 
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

In [7]:
# 다차원 배열에 적용가능한 소프트맥스 함수 정의
def softmax(x):
    x = x.T    # x.ndim==2 일때도 일관성있기 처리하기 위해 transpose를 취한다.
    exp_x = np.exp(x - np.max(x, axis=0))
    sum_exp_x = np.sum(exp_x, axis=0)
    y = exp_x / sum_exp_x
    return y.T

# 미니배치 학습을 위한 교차 엔트로피 오차
def cross_entropy_error(y, t):
    # mini-batch size가 1일때도 일관성이 있도록 모양을 2차원으로 다시 잡아준다.
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    delta = 1e-7

    if one_hot_label:
        return -np.sum(t * np.log(y + delta)) / batch_size
    # one_hot_label=False일 때, 예측 y는 여전히 one_hot_encoding이지만 레이블 t는 2, 7과 같은 숫자가 된다.
    else:
        return -np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size

In [8]:
# Softmax-with-Loss 계층
# 이 계층의 (역전파)출력값은 Softmax의 (순전파)출력값과 정답 레이블(one-hot encoding)의 차분이다.
# 회귀(regression)도 결과가 위와 같다.
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실함수
        self.y = None     # softmax의 출력
        self.t = None     # 정답 레이블(원-핫 인코딩 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:  # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        return dx